In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import warnings; warnings.simplefilter('ignore')

In [ ]:
jobs = pd.read_csv('jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
jobs.head()

In [ ]:
users = pd.read_csv('users.tsv' ,delimiter='\t',encoding='utf-8')
users.head()

In [ ]:
user_history = pd.read_csv('user_history.tsv', delimiter='\t',encoding='utf-8')
user_history.head()

In [ ]:
apps = pd.read_csv('apps.tsv', delimiter='\t',encoding='utf-8')
apps.head()

In [ ]:
test_users = pd.read_csv('test_users.tsv', delimiter='\t',encoding='utf-8')
test_users.head()

In [ ]:
user_history_training = user_history.loc[user_history['Split'] =='Train']
user_history_testing = user_history.loc[user_history['Split'] =='Test']
apps_training = apps.loc[apps['Split'] == 'Train']
apps_testing = apps.loc[apps['Split'] == 'Test']
users_training = users.loc[users['Split']=='Train']
users_testing = users.loc[users['Split']=='Test']

In [ ]:
apps_training.shape

In [ ]:
apps_training.head()

In [ ]:
apps_testing.head()

In [ ]:
apps_testing.shape

In [ ]:
user_history_training.shape

In [ ]:
user_history_training.head()

In [ ]:
user_history_testing.shape

In [ ]:
user_history_testing.head()

In [ ]:
users_training.shape

In [ ]:
users_training.head()

In [ ]:
users_testing.shape

In [ ]:
users_testing.head()

In [ ]:
jobs.groupby(['City','State','Country']).size().reset_index(name='Locationwise')

In [ ]:
jobs.groupby(['Country']).size().reset_index(name='Locationwise').sort_values('Locationwise', ascending=False).head()

In [ ]:
Country_wise_job = jobs.groupby(['Country']).size().reset_index(name='Locationwise').sort_values('Locationwise', ascending=False)

In [ ]:
plt.figure(figsize=(12,12))
ax = sns.barplot(x="Country", y="Locationwise", data=Country_wise_job)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
ax.set_title('Country wise job openings')
plt.tight_layout()
plt.show()

In [ ]:
jobs_US = jobs.loc[jobs['Country']=='US']

In [ ]:
jobs_US[['City','State','Country']]

In [ ]:
jobs_US.groupby(['City','State','Country']).size().reset_index(name='Locationwise').sort_values('Locationwise', ascending=False).head())

In [ ]:
State_wise_job_US = jobs_US.groupby(['State']).size().reset_index(name='Locationwise').sort_values('Locationwise',ascending=False)

In [ ]:
State_wise_job_US

In [ ]:
plt.figure(figsize=(12,12))
ax = sns.barplot(x="State", y="Locationwise",data=State_wise_job_US)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
ax.set_title('State wise job openings')
plt.tight_layout()
plt.show()

In [ ]:
jobs_US.groupby(['City']).size().reset_index(name='Locationwise').sort_values('Locationwise',ascending=False)

In [ ]:
City_wise_location = jobs_US.groupby(['City']).size().reset_index( name='Locationwise').sort_values('Locationwise',ascending=False)

In [ ]:
City_wise_location_th = City_wise_location.loc[City_wise_location['Locationwise']>=12]

In [ ]:
City_wise_location_th

In [ ]:
plt.figure(figsize=(12,12))
ax = sns.barplot(x="City", y="Locationwise",data=City_wise_location_th.head(50))
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
ax.set_title('City wise job openings')
plt.tight_layout()
plt.show()

In [ ]:
users_training.groupby(['Country']).size().reset_index(name='Locationwise').sort_values('Locationwise', ascending=False).head()

In [ ]:
user_training_US = users_training.loc[users_training['Country']=='US']

In [ ]:
user_training_US.shape

In [ ]:
user_training_US.groupby(['State']).size().reset_index( name='Locationwise_state').sort_values('Locationwise_state',ascending=False)

In [ ]:
user_training_US_state_wise = user_training_US.groupby(['State']).size().reset_index( name='Locationwise_state').sort_values('Locationwise_state',ascending=False)

In [ ]:
user_training_US_th = user_training_US_state_wise.loc[user_training_US_state_wise['Locationwise_state']>=12]

In [ ]:
plt.figure(figsize=(12,12))
ax = sns.barplot(x="State", y="Locationwise_state",data=user_training_US_th.head(50))
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
ax.set_title('State wise job seekers')
plt.tight_layout()
plt.show()

In [ ]:
user_training_US.groupby(['City']).size().reset_index( name='Locationwise_city').sort_values('Locationwise_city',ascending=False)

In [ ]:
user_training_US_city_wise = user_training_US.groupby(['City']).size().reset_index( name='Locationwise_city').sort_values('Locationwise_city',ascending=False)

In [ ]:
user_training_US_City_th = user_training_US_city_wise.loc[user_training_US_city_wise['Locationwise_city']>=12]

In [ ]:
plt.figure(figsize=(12,12))
ax = sns.barplot(x="City", y="Locationwise_city",data=user_training_US_City_th.head(50))
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
ax.set_title('State wise job seekers')
plt.tight_layout()
plt.show()

In [ ]:
jobs_US.columns

In [ ]:
jobs_US.head().transpose()

In [ ]:
jobs_US_base_line = jobs_US.iloc[0:10000,0:8]

In [ ]:
jobs_US_base_line.head()

In [ ]:
jobs_US_base_line['Title'] = jobs_US_base_line['Title'].fillna('')
jobs_US_base_line['Description'] = jobs_US_base_line['Description'].fillna('')
jobs_US_base_line['Description'] = jobs_US_base_line['Title'] + jobs_US_base_line['Description']

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(jobs_US_base_line['Description'])

In [ ]:
tfidf_matrix.shape

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[0]

In [ ]:
user_based_approach = user_based_approach.reset_index()
userid = user_based_approach['UserID']
indices = pd.Series(user_based_approach.index, index=user_based_approach['UserID'])

In [ ]:
def get_recommendations_userwise(userid):
    idx = indices[userid]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    user_indices = [i[0] for i in sim_scores]
    
    return user_indices[0:6]

In [ ]:
def get_job_id(usrid_list):
    jobs_userwise = apps_training['UserID'].isin(usrid_list)
    df1 = pd.DataFrame(data = apps_training[jobs_userwise], columns=['JobID'])
    joblist = df1['JobID'].tolist()
    Job_list = jobs['JobID'].isin(joblist)
    df_temp = pd.DataFrame(data = jobs[Job_list], columns=['JobID','Title','Description','City','State'])
    return df_temp

In [ ]:
get_recommendations_userwise(47)

In [ ]:
get_job_id(get_recommendations_userwise(47))